In [ ]:
import ai_flow as af
from ai_flow.project.project_config import ProjectConfig
from ai_flow.workflow.job_config import JobConfig
from ai_flow.workflow.workflow_config import WorkflowConfig
from ai_flow_plugins.job_plugins.bash import BashProcessor


def main():
    project_config: ProjectConfig = ProjectConfig()
    project_config.set_project_name('simple_workflow')
    project_config.set_server_uri('localhost:50051')
    project_config.set_notification_server_uri('localhost:50052')
    project_config['blob'] = {
        'blob_manager_class': 'ai_flow_plugins.blob_manager_plugins.local_blob_manager.LocalBlobManager',
        'blob_manager_config': {
                'root_directory': '/tmp/aiflow-examples/notebook'
            }
    }

    workflow_config: WorkflowConfig = WorkflowConfig('simple_workflow')
    workflow_config.add_job_config('task_1', JobConfig(job_name='task_1', job_type='bash'))
    workflow_config.add_job_config('task_2', JobConfig(job_name='task_2', job_type='bash'))

    af.init_notebook_context(project_config, workflow_config)

    with af.job_config('task_1'):
        af.user_define_operation(processor=BashProcessor("echo hello"))
    with af.job_config('task_2'):
        af.user_define_operation(processor=BashProcessor("echo hello"))

    af.action_on_job_status('task_2', 'task_1')

    workflow_name = af.current_workflow_config().workflow_name
    stop_workflow_executions(workflow_name)
    af.workflow_operation.submit_workflow(workflow_name)
    af.workflow_operation.start_new_workflow_execution(workflow_name)


def stop_workflow_executions(workflow_name):
    workflow_executions = af.workflow_operation.list_workflow_executions(workflow_name)
    for workflow_execution in workflow_executions:
        af.workflow_operation.stop_workflow_execution(workflow_execution.workflow_execution_id)


if __name__ == '__main__':
    main()